In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/visdrone2019DET.zip" -d "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/"

In [2]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 8.4 MB/s 


In [3]:
import argparse
import sys
sys.path.append('/content/drive/MyDrive/MobileCount/CrowdCounting-on-VisDrone2020-main/src')

from ast import literal_eval
from callbacks import call_dict
from evaluate import evaluate_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from models.CC import CrowdCounter
from dataset.visdrone import load_test, load_train_val, cfg_data
from dataset.run_datasets import make_dataset
from run import run_model, run_transforms
from train import Trainer
from config import cfg
import numpy as np
import torch

In [4]:
def load_CC_train():
    """
    Load CrowdCounter model net for training mode
    """
    cc = CrowdCounter([0], cfg.NET)
    return cc

def load_CC_test():

    """
    Load CrowdCounter model net for testing mode
    """
    cc = CrowdCounter([0], cfg.NET)
    if cfg.PRE_TRAINED:
        cc.load(cfg.PRE_TRAINED)
    return cc

# Carica modello pre-trained di Visdrone Object Detection per il fine-tuning di Visdrone Crowd Counting
def load_OD_pretrained_train():
  PATH = '/content/drive/MyDrive/MobileCount/CrowdCounting-on-VisDrone2020-main/exp/08-25_11-59_VisDrone_MobileCountx0_5_0.001__540x960_NVS_OBJECT_DETECTION_LR1E-3/all_ep_32_mae_10.0_rmse_16.3.pth'
  model = CrowdCounter([0], cfg.NET)
  checkpoint = torch.load(PATH)
  model.load_state_dict(checkpoint['model_state_dict'])

  print('Modello pre-trained caricato con successo!')
 
  return model

In [5]:
def test_net():
    """
    Test a model on a specific test set
    Must specify the function tha returns the model and the dataset
    """
    res = evaluate_model(model_function=load_CC_test,
                         data_function=load_test,
                         bs=cfg.TEST_BATCH_SIZE,
                         n_workers=cfg.N_WORKERS,
                         losses={'rmse': lambda x, y: mean_squared_error(x, y, squared=False),
                                 'mae': mean_absolute_error},
                         out_prediction=None
                         )
    print(res)


def run_net(in_file, callbacks):
    """
    Run the model on a given file or folder

    @param in_file: media file or folder of images
    @param callbacks: list of callbacks to be called after every forward operation
    """
    dataset = make_dataset(in_file)

    transforms = run_transforms(cfg_data.MEAN, cfg_data.STD, cfg_data.SIZE)
    dataset.set_transforms(transforms)

    callbacks_list = [(call_dict[call] if type(call) == str else call) for call in callbacks]

    run_model(load_CC_test, dataset, cfg.TEST_BATCH_SIZE, cfg.N_WORKERS, callbacks_list)


def train_net():
    """
    Train the given model on a given data loader
    """
    trainer = Trainer(dataloader=load_train_val,
                      cfg_data=cfg_data,
                      net_fun=load_CC_train
                      )
    
    trainer.train()


def train_net_pretrained():
    """
    Train the given model on a given data loader
    """
    trainer = Trainer(dataloader=load_train_val,
                      cfg_data=cfg_data,
                      net_fun=load_OD_pretrained_train
                      )
    
    trainer.train()

In [ ]:
if __name__ == '__main__':
    seed = cfg.SEED
    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)

    parser = argparse.ArgumentParser(description='Execute a training, an evaluation or run the net on some example')
    parser.add_argument('mode', type=str, help='can be train, test or run')
    parser.add_argument('--path', type=str, help='in run mode, the input file or folder to be processed')
    parser.add_argument('--callbacks', type=str,
                        help='List of callbacks, they can be [\'save_callback\', \'count_callback\']')
    #args = parser.parse_args()
    train_net()
    #train_net_pretrained()
    #test_net()

''' if args.callbacks is not None:
        callbacks = literal_eval(args.callbacks)
    else:
        callbacks = []
    if args.mode == 'train':
        train_net()
    elif args.mode == 'test':
        test_net()
    elif args.mode == 'run':
        run_net(args.path, callbacks)'''

    

In [11]:
#TAKE GT COUNT
import h5py
import torch
from tqdm import tqdm
from dataset.visdrone import load_test, load_train_val, cfg_data
import cv2

train_loader, val_loader = load_train_val()

tk_train = tqdm(
            enumerate(train_loader, 0), total=len(train_loader), leave=False,bar_format='{l_bar}{bar:32}{r_bar}',
            colour='#ff0de7')
        
for i, data in tk_train:
    img, gt = data
    gt = gt.to('cuda')
    img = img.to('cuda')

    norm_gt_count = torch.mean(torch.sum(gt, dim=(1, 2))).data / 2550.0
    print('gt count: ')
    print(norm_gt_count.item())

Visdrone train = True
Visdrone img_transform = True
Visdrone gt_transform = True
Visdrone train = True
Visdrone img_transform = True
Visdrone gt_transform = True


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
                                                                   

gt count: 
207.60107421875
